In [20]:
import os
import json
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm
import albumentations as A
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import GroupKFold

In [6]:
IMAGE_ROOT = "/opt/ml/input/data/train/DCM"
LABEL_ROOT = "/opt/ml/input/data/train/outputs_json"

CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]

CLASS2IND = {v: i for i, v in enumerate(CLASSES)}
IND2CLASS = {v: k for k, v in CLASS2IND.items()}

pngs = {
    os.path.relpath(os.path.join(root, fname), start=IMAGE_ROOT)
    for root, _dirs, files in os.walk(IMAGE_ROOT)
    for fname in files
    if os.path.splitext(fname)[1].lower() == ".png"
}
pngs = sorted(pngs)

In [22]:
temp_mean, temp_std = [], []

for png in tqdm(pngs, desc="Calculate Image Mean & Std..."):
    image_path = os.path.join(IMAGE_ROOT, png)
    image = cv2.imread(image_path)

    per_mean = image.mean(axis=(0, 1))
    per_std = (image ** 2).mean(axis=(0, 1))
    temp_mean.append(per_mean)
    temp_std.append(per_std)

print()
global_mean = (np.mean(temp_mean, axis=0)) / 255
global_std = (np.mean(temp_std, axis=0) - global_mean ** 2) ** 0.5 / 255

Calculate Image Mean & Std...: 100%|██████████| 800/800 [04:10<00:00,  3.19it/s]


Global Mean = [0.12099939 0.12099939 0.12099939]
Global Std = [0.0363468 0.0363468 0.0363468]


In [23]:
print(f"Global Mean = {global_mean}")
print(f"Global Std = {global_std}")

Global Mean = [0.12099939 0.12099939 0.12099939]
Global Std = [0.0363468 0.0363468 0.0363468]


In [19]:
## Mean & Std Calculation for Albumentations.Normalize

